In [1]:
#pip install geopy

In [2]:
#pip install geopandas

In [3]:
#pip install xlrd

In [4]:
import pandas as pd
import numpy as np
import json
from geopy.distance import geodesic 
import geopandas as gpd
import math

/Users/polinaminkovski/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
df = pd.read_csv("archive data files/Blocks Data - DD_sample_data.csv")

In [6]:
df.shape

(2127, 71)

In [7]:
df_zips = pd.read_excel("ZIP_Locale_Detail.xls")
#https://postalpro.usps.com/ZIP_Locale_Detail

In [8]:
#import the county-zip mapping file (US Census)
#df_county = gpd.read_file("PVS_23_v2_ccd_12011.shp")

In [9]:
df_lon_lat = pd.read_csv("uszips.csv")
#https://simplemaps.com/data/us-zips

Streamline Zip Codes for location mapping

In [10]:
#make sure all zips are 5 digit strings
def zips(i):
    y = str(i).rstrip('.0')
    if len(y) == 3:
        x = ''.join(("00",y))
    elif len(y) == 4:
        x = ''.join(("0",y))
    elif len(y) == 2:
        x = ''.join((y,"000"))
    elif len(y) == 1:
        x = ''.join((y,"0000"))
    else:
        x = y
    return x

In [11]:
#make sure all zips in blocks data file are 5 digits (will append 0s to the end, since these are all Florida zips)
def zips_(i):
    y = str(i).rstrip('.0')
    if len(y) == 3:
        x = ''.join((y,"00"))
    elif len(y) == 4:
        x = ''.join((y,"0"))
    elif len(y) == 2:
        x = ''.join((y,"000"))
    elif len(y) == 1:
        x = ''.join((y,"0000"))
    else:
        x = y
    return str(x)

In [12]:
df_zips["PHYSICAL ZIP_CHANGE"] = df_zips["PHYSICAL ZIP"].apply(lambda x: zips(x))

In [13]:
df['voting_zip_change']=df['voting_zipcode'].apply(lambda x: zips_(x))

In [14]:
df['collection_location_zip_change']=df['collection_location_zip'].apply(lambda x: zips_(x) )

In [15]:
#convert extras from string to dictionary
df['extras'] = df['extras'].apply(lambda x: json.loads(x))

Create location fields for mapping and calculate distance between clean voting and collection location fields

In [16]:
def concat(x,y):
    if pd.isna(x):
        x = 0
    if pd.isna(y):
        y = 0
    return f"{x},{y}"

In [17]:
#create a lat/lon field in the reference file
df_lon_lat['location']=df_lon_lat.apply(lambda row: concat(row['lat'], row['lng']), axis=1)

In [18]:
df['voting_location'] = df.apply(lambda row: concat(row['voting_address_latitude'], row['voting_address_longitude']), axis=1)

In [19]:
df_lon_lat['zip']=df_lon_lat['zip'].astype(str)

In [20]:
df = pd.merge(df,df_lon_lat[['zip','location']],left_on='voting_zip_change',right_on="zip")

In [21]:
df.columns

Index(['voter_registration_scan_id', 'upload_time',
       'voter_registration_scan_updated_at', 'visual_qc_county',
       'is_registration_form', 'registration_form_id', 'registration_type',
       'first_name', 'middle_name', 'last_name', 'name_suffix',
       'voting_street_address_one', 'voting_street_address_two', 'voting_city',
       'voting_state', 'voting_zipcode', 'mailing_street_address_one',
       'mailing_street_address_two', 'mailing_city', 'mailing_zipcode',
       'data_entry_county', 'gender', 'date_of_birth', 'identification',
       'phone_number', 'email_address', 'eligible_voting_age', 'signature',
       'extras', 'data_entry_time', 'data_entry_updated_at', 'party',
       'name_prefix', 'ethnicity', 'voting_address_latitude',
       'voting_address_longitude', 'distance_from_location',
       'address_validated', 'shift_id', 'shift_start', 'shift_end',
       'field_start', 'field_end', 'visual_qc_completed_by_user_id',
       'phone_verification_completed_by_u

In [22]:
#map lat/lon for missing lat/lon fields in voting location
df['voting_location_modified']=np.where(df['voting_location']=='0,0',df['location'],df['voting_location'])

In [23]:
df['collection_location'] = df.apply(lambda row: concat(row['collection_location_latitude'], row['collection_location_longitude']), axis=1)

In [24]:
def dist(x, y):
    return round(geodesic(eval(x),eval(y)).miles,1)


In [25]:
#extract dictionary into df with separate columns for each key
df_extras = df.extras.apply(pd.Series)

In [26]:
#merge original df with dictionary df
df_clean = pd.merge(right=df, left=df_extras, right_index = True, left_index=True)

In [27]:
#merge with streamlined zip codes for voting location
df_clean_location = pd.merge(left=df_clean, right=df_zips[["PHYSICAL ZIP_CHANGE", "PHYSICAL CITY"]].drop_duplicates(subset=['PHYSICAL ZIP_CHANGE']), left_on = "voting_zip_change", right_on = "PHYSICAL ZIP_CHANGE",how='left')

In [28]:
df_clean_location.rename(columns={"PHYSICAL CITY": "voting_city_clean"}, inplace=True)

In [29]:
df_clean_location = df_clean_location.drop(["PHYSICAL ZIP_CHANGE"], axis=1)

In [30]:
#merge with streamlined zip codes for collection location
df_clean_location_ = pd.merge(left=df_clean_location, right=df_zips[["PHYSICAL ZIP_CHANGE", "PHYSICAL CITY"]].drop_duplicates(subset=['PHYSICAL ZIP_CHANGE']), left_on = "collection_location_zip_change", right_on = "PHYSICAL ZIP_CHANGE", how="left")

In [31]:
df_clean_location_.rename(columns={"PHYSICAL CITY": "collection_city_clean"}, inplace=True)

In [32]:
df_clean_location_["distance"] = df_clean_location_.apply(lambda row: dist(row['voting_location_modified'], row['collection_location']), axis=1)

Clean up disparate versions for all dem fields extracted from Extras

In [33]:
#combine gender columns
gender_columns = [col for col in df_clean_location_.columns if 'gender' in col]

In [34]:
df_clean_location_['gender_combined'] = df_clean_location_[gender_columns].bfill(axis=1).iloc[:, 0]

In [35]:
#combine race columns
race_columns = [col for col in df_clean_location_.columns if 'race' in col]

In [36]:
df_clean_location_['race_combined']=df_clean_location_[race_columns].bfill(axis=1).iloc[:, 0]

In [37]:
#combine birth location columns
birth_combined = [col for col in df_clean_location_.columns if 'birth' in col]

In [38]:
df_clean_location_['birthplace_combined']=df_clean_location_[birth_combined].bfill(axis=1).iloc[:, 0]

In [39]:
df_clean_location_['gender']=df_clean_location_['gender_combined']
df_clean_location_['race and ethnicity'] = df_clean_location_['race_combined']
df_clean_location_['birthplace'] = df_clean_location_['birthplace_combined']
df_clean_location_.drop(columns=['gender_combined', 'race_combined','birthplace_combined'], inplace=True)

Clean up County Columns

In [40]:
#create a clean lookup file (Zip Codes to Counties)
#https://www.unitedstateszipcodes.org/fl/
df_zips_counties = pd.read_csv("Zips and Counties - Sheet1.csv", header=None, names=["data"])


In [41]:
df_zips_counties['zips_filter']=df_zips_counties['data'].str.isnumeric()
df_zips_counties['counties_filter']=df_zips_counties['data'].str.contains(" County")

zips = df_zips_counties[df_zips_counties['zips_filter']==True]
counties = df_zips_counties[df_zips_counties['counties_filter']==True]


In [42]:
# combine zips and counties 
zips.reset_index(drop=True, inplace=True)
counties.reset_index(drop=True, inplace=True)
df_zips_counties_combined = pd.concat([zips,counties], axis=1)

In [43]:
df_zips_counties_combined.columns.values[0]='zip_code'
df_zips_counties_combined.columns.values[3]='county'

In [44]:
#df_zips_counties_combined['zip_code']=df_zips_counties_combined['zip_code'].astype(float)

In [45]:
#apply clean zip code file to full dataset
df_clean_location__ = pd.merge(left=df_clean_location_, right=df_zips_counties_combined, left_on = "voting_zip_change", right_on = "zip_code",how='left')

In [46]:
df_clean_location__.rename(columns={"county": "voting_county"}, inplace=True)
df_clean_location__.rename(columns={'PHYSICAL ZIP_CHANGE':'PHYSICAL_ZIP'}, inplace=True)

In [47]:
df_clean_location__['voting_street_address_one'] = df_clean_location__['voting_county']
df_clean_location__['voting_zipcode'] = df_clean_location__['voting_zip_change']
df_clean_location__['voting_location'] = df_clean_location__['voting_location_modified']
df_clean_location__['collection_location_zip'] = df_clean_location__['collection_location_zip_change']

df_clean_location__.drop(columns=[
    'zip_code', 'zips_filter',
        'counties_filter', 'voting_county', 'zips_filter', 'counties_filter','zip_x','location_x',
        'zip_y','location_y','voting_location_modified','collection_location_zip_change','voting_zip_change'], inplace=True)

In [48]:
#create csv
df_clean_location__.to_csv("clean_blocks_.csv", index=False)